##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Data Validation

***Un ejemplo de un componente clave de TensorFlow Extended***

Nota: Puede ejecutar este ejemplo ahora mismo en un bloc de notas estilo Jupyter, ¡no es necesario configurarlo! Simplemente haga clic en "Ejecutar en Google Colab"

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/data_validation/tfdv_basic.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/data_validation/tfdv_basic.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/data_validation/tfdv_basic.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table></div>

Este bloc de notas de Colab de ejemplo ilustra cómo se puede usar TensorFlow Data Validation (TFDV) para investigar y visualizar su conjunto de datos. Eso incluye observar estadísticas descriptivas, inferir un esquema, verificar y corregir anomalías, y verificar desviaciones y sesgos en nuestro conjunto de datos. Es importante comprender las características de su conjunto de datos, incluido cómo podría cambiar con el tiempo en su canalización de producción. También es importante buscar anomalías en sus datos y comparar sus conjuntos de datos de entrenamiento, evaluación y servicio para asegurarse de que sean consistentes.

Usaremos datos del [conjunto de datos Taxi Trips](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) publicado por la ciudad de Chicago.

Nota: Este sitio ofrece aplicaciones que usan datos que fueron modificados para su uso desde su fuente original, www.cityofchicago.org, el sitio web oficial de la ciudad de Chicago. La ciudad de Chicago no garantiza el contenido, la exactitud, la puntualidad o la integridad de ninguno de los datos que se proporcionan en este sitio. Los datos proporcionados en este sitio están sujetos a cambios en cualquier momento. Se entiende que los datos proporcionados en este sitio se usan bajo su propia responsabilidad.

[Más información](https://cloud.google.com/bigquery/public-data/chicago-taxi) sobre el conjunto de datos en [Google BigQuery](https://cloud.google.com/bigquery/). Explore el conjunto de datos completo en la [interfaz de usuario de BigQuery](https://bigquery.cloud.google.com/dataset/bigquery-public-data:chicago_taxi_trips).

Punto clave: Como modelador y desarrollador, piense en cómo se usan estos datos y los posibles beneficios y daños que pueden causar las predicciones de un modelo. Un modelo como este podría reforzar los prejuicios y las disparidades sociales. ¿Es una característica relevante para el problema que desea resolver o introducirá un sesgo? Para obtener más información, lea sobre [la equidad del aprendizaje automático](https://developers.google.com/machine-learning/fairness-overview/).

Las columnas del conjunto de datos son las siguientes:

<table>
<tr>
<td>pickup_community_area</td>
<td>fare</td>
<td>trip_start_month</td>
</tr>
<tr>
<td>trip_start_hour</td>
<td>trip_start_day</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>pickup_latitude</td>
<td>pickup_longitude</td>
<td>dropoff_latitude</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>trip_miles</td>
<td>pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>payment_type</td>
<td>company</td>
</tr>
<tr>
<td>trip_seconds</td>
<td>dropoff_community_area</td>
<td>tips</td>
</tr>
</table>

## Instalación e importación de paquetes

Instale los paquetes para TensorFlow Data Validation.

### Actualización de pip

Para evitar actualizar Pip en un sistema cuando se ejecuta localmente, verifique que se esté ejecutando en Colab. Por supuesto, los sistemas locales se pueden actualizar por separado.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instalación de paquetes de Data Validation

Instale los paquetes y las dependencias de TensorFlow Data Validation, lo que demora unos minutos. Es posible que vea advertencias y errores relacionados con versiones de dependencias incompatibles, que se resolverá en la siguiente sección.

In [ ]:
print('Installing TensorFlow Data Validation')
!pip install --upgrade 'tensorflow_data_validation[visualization]<2'

### Importación de TensorFlow y recarga de paquetes actualizados

En el paso anterior se actualizan los paquetes predeterminados en el entorno de Gooogle Colab, por lo que se deben recargar los recursos del paquete para resolver las nuevas dependencias.

Nota: En este paso se resuelve el error de dependencia de la instalación. Si sigue teniendo problemas de ejecución de código después de ejecutar este código, reinicie el tiempo de ejecución (Tiempo de ejecución &gt; Reiniciar tiempo de ejecución...).

In [ ]:
import pkg_resources
import importlib
importlib.reload(pkg_resources)

Verifique las versiones de TensorFlow y Data Validation antes de continuar. 

In [ ]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
print('TF version:', tf.__version__)
print('TFDV version:', tfdv.version.__version__)

## Carga del conjunto de datos

Descargaremos nuestro conjunto de datos de Google Cloud Storage.

In [ ]:
import os
import tempfile, urllib, zipfile

# Set up some globals for our file paths
BASE_DIR = tempfile.mkdtemp()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'chicago_taxi_output')
TRAIN_DATA = os.path.join(DATA_DIR, 'train', 'data.csv')
EVAL_DATA = os.path.join(DATA_DIR, 'eval', 'data.csv')
SERVING_DATA = os.path.join(DATA_DIR, 'serving', 'data.csv')

# Download the zip file from GCP and unzip it
zip, headers = urllib.request.urlretrieve('https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/chicago_data.zip')
zipfile.ZipFile(zip).extractall(BASE_DIR)
zipfile.ZipFile(zip).close()

print("Here's what we downloaded:")
!ls -R {os.path.join(BASE_DIR, 'data')}

## Cálculo y visualización de estadísticas

Primero usaremos [`tfdv.generate_statistics_from_csv`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/generate_statistics_from_csv) para calcular estadísticas para nuestros datos de entrenamiento. (Ignore las breves advertencias)

TFDV puede calcular [estadísticas](https://github.com/tensorflow/metadata/blob/v0.6.0/tensorflow_metadata/proto/v0/statistics.proto) descriptivas que brindan una descripción general rápida de los datos en términos de las características presentes y las formas de sus distribuciones de valores.

Internamente, TFDV usa el marco de procesamiento de datos paralelo de [Apache Beam](https://beam.apache.org/) para escalar el cálculo de estadísticas en grandes conjuntos de datos. Para las aplicaciones que desean una integración más profunda con TFDV (por ejemplo, adjuntar la generación de estadísticas al final de una canalización de generación de datos), la API también expone un Beam PTransform para la generación de estadísticas.

In [ ]:
train_stats = tfdv.generate_statistics_from_csv(data_location=TRAIN_DATA)

Ahora usemos [`tfdv.visualize_statistics`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/visualize_statistics), que usa [Facets](https://pair-code.github.io/facets/) para crear una visualización concisa de nuestros datos de entrenamiento:

- Tenga en cuenta que las características numéricas y las características categóricas se visualizan por separado y que se muestran gráficos donde se representan las distribuciones de cada característica.
- Tenga en cuenta que las características con valores faltantes o cero muestran un porcentaje en rojo como indicador visual de que puede haber problemas con los ejemplos de esas características. El porcentaje es el porcentaje de ejemplos a los que les faltan valores o tienen valores cero para esa característica.
- Tenga en cuenta que no hay ejemplos con valores para `pickup_census_tract`. ¡Esta es una oportunidad para reducir la dimensionalidad!
- Intente hacer clic en "expandir", ubicado arriba de los gráficos, para cambiar la visualización.
- Pruebe pasar el cursor sobre las barras en los gráficos para que se muestren los rangos y los recuentos de los cubos.
- Pruebe cambiar entre las escalas logarítmica y lineal y observe cómo la escala logarítmica revela muchos más detalles sobre la característica categórica `payment_type`
- Intente seleccionar "cuantiles" en el menú "Gráfico para mostrar" y coloque el cursor sobre los marcadores para ver los porcentajes de cuantiles.

In [ ]:
# docs-infra: no-execute
tfdv.visualize_statistics(train_stats)

<!-- <img class="tfo-display-only-on-site" src="images/statistics.png"/> -->

## Inferencia de un esquema

Ahora usemos [`tfdv.infer_schema`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/infer_schema) para crear un esquema para nuestros datos. Un esquema define restricciones para los datos que son relevantes para el aprendizaje automático. Las restricciones de ejemplo incluyen el tipo de datos de cada característica, ya sea numérica o categórica, o la frecuencia de su presencia en los datos. Para características categóricas, el esquema también define el dominio: la lista de valores aceptables. Dado que escribir un esquema puede ser una tarea tediosa, especialmente para conjuntos de datos con muchas características, TFDV proporciona un método para generar una versión inicial del esquema basada en estadísticas descriptivas.

Es importante que el esquema sea correcto porque el resto de la cadena de producción dependerá de que el esquema generado por TFDV lo sea.  El esquema también aporta documentación para los datos, por lo que es útil cuando diferentes desarrolladores trabajan con los mismos datos. Usemos [`tfdv.display_schema`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/display_schema) para mostrar el esquema inferido y poder revisarlo.

In [ ]:
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=schema)

## Verificación de los datos de evaluación para detectar errores

Hasta ahora, solo analizamos los datos de entrenamiento.  Es importante que nuestros datos de evaluación sean coherentes con nuestros datos de entrenamiento, y que utilicen el mismo esquema.  También es importante que los datos de evaluación incluyan ejemplos de aproximadamente los mismos rangos de valores para nuestras características numéricas que nuestros datos de entrenamiento, para que nuestra cobertura de la superficie de pérdida durante la evaluación sea aproximadamente la misma que durante el entrenamiento.  Lo mismo ocurre con las características categóricas. De lo contrario, podríamos tener problemas de entrenamiento que no se identifican durante la evaluación, porque no evaluamos parte de nuestra superficie de pérdidas.

- Observe que cada característica ahora incluye estadísticas para los conjuntos de datos de entrenamiento y evaluación.
- Observe que los gráficos ahora tienen los conjuntos de datos de entrenamiento y evaluación que se superponen, lo que facilita su comparación.
- Observe que los gráficos ahora incluyen una vista de porcentajes, que se puede combinar con escalas logarítmicas o lineales predeterminadas.
- Observe que la media y la mediana de `trip_miles` son diferentes para los conjuntos de datos de entrenamiento y de evaluación. ¿Eso causará problemas?
- Bueno, los valores `tips` máximos son muy diferentes para los conjuntos de datos de entrenamiento y de evaluación. ¿Eso causará problemas?
- Haga clic en expandir en el gráfico Características numéricas y seleccione la escala logarítmica. Revise la característica `trip_seconds` y observe la diferencia en el máximo. ¿La evaluación no tendrá en cuenta partes de la superficie de pérdida?

In [ ]:
# Compute stats for evaluation data
eval_stats = tfdv.generate_statistics_from_csv(data_location=EVAL_DATA)

In [ ]:
# docs-infra: no-execute
# Compare evaluation data with training data
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

<!-- <img class="tfo-display-only-on-site" src="images/statistics_eval.png"/> -->

## Verificación de anomalías en la evaluación

¿El conjunto de datos de evaluación coincide con el esquema del conjunto de datos de entrenamiento?  Esto es especialmente importante para las características categóricas, en las que queremos identificar el rango de valores aceptables.

Punto clave: ¿Qué pasaría si intentáramos usar datos con valores de características categóricas que no estuvieran en nuestro conjunto de datos de entrenamiento para ejecutar la evaluación? ¿Qué pasa con las características numéricas que están fuera de los rangos de nuestro conjunto de datos de entrenamiento?

In [ ]:
# Check eval data for errors by validating the eval data stats using the previously inferred schema.
anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)
tfdv.display_anomalies(anomalies)

## Corrección de anomalías de evaluación en el esquema

¡Ups! Parece que hay algunos valores nuevos para `company` en nuestros datos de evaluación, que no teníamos en nuestros datos de entrenamiento. También tenemos un nuevo valor para `payment_type`. Éstas deben considerarse anomalías, pero lo que decidamos hacer al respecto depende de nuestro conocimiento de los datos. Si una anomalía realmente indica un error de datos, entonces los datos subyacentes deben corregirse. De lo contrario, simplemente podemos actualizar el esquema para incluir los valores en el conjunto de datos de evaluación.

Punto clave: Si no solucionáramos estos problemas, ¿cómo se verían afectados los resultados de la evaluación?

A menos que cambiemos nuestro conjunto de datos de evaluación, no podemos arreglarlo todo, pero podemos arreglar algunas cosas en el esquema que nos resultan cómodas de aceptar. Eso incluye flexibilizar nuestra visión de lo que es y lo que no es una anomalía en determinadas características, así como actualizar nuestro esquema para que incluya valores faltantes en las características categóricas. TFDV nos permite detectar lo que debemos arreglar.

Hagamos esas correcciones ahora y luego repasemos una vez más.

In [ ]:
# Relax the minimum fraction of values that must come from the domain for feature company.
company = tfdv.get_feature(schema, 'company')
company.distribution_constraints.min_domain_mass = 0.9

# Add new value to the domain of feature payment_type.
payment_type_domain = tfdv.get_domain(schema, 'payment_type')
payment_type_domain.value.append('Prcard')

# Validate eval stats after updating the schema 
updated_anomalies = tfdv.validate_statistics(eval_stats, schema)
tfdv.display_anomalies(updated_anomalies)

¡Miren eso! ¡Verificamos que los datos de entrenamiento y evaluación ahora son coherentes! Gracias, TFDV ;)

## Entornos de esquema

También dividimos un conjunto de datos de "servicio" para este ejemplo, por lo que también deberíamos verificarlo. De forma predeterminada, todos los conjuntos de datos de una canalización deben usar el mismo esquema, pero suele haber excepciones. Por ejemplo, en el aprendizaje supervisado tenemos que incluir etiquetas en nuestro conjunto de datos, pero cuando servimos el modelo para inferencia, no se incluyen las etiquetas. En algunos casos es necesario introducir ligeras variaciones en el esquema.

Los **entornos** se pueden usar para expresar dichos requisitos. En particular, las características del esquema se pueden asociar a un conjunto de entornos a través de `default_environment`, `in_environment` y `not_in_environment`.

Por ejemplo, en este conjunto de datos, la característica `tips` se incluye como etiqueta para el entrenamiento, pero falta en los datos de servicio. Sin que se especifique un entorno, se mostrará como una anomalía.

In [ ]:
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

Nos ocuparemos de la característica `tips` más adelante. También tenemos un valor INT en nuestros segundos de viaje, donde nuestro esquema esperaba un FLOAT. Al notar esa diferencia, TFDV ayuda a descubrir inconsistencias en la forma en que se generan los datos para el entrenamiento y el servicio. Es muy fácil no darse cuenta de problemas como ese hasta que el rendimiento del modelo se ve afectado, a veces de forma catastrófica. Puede que se trate de un problema importante o puede que no, pero en cualquier caso se debería investigar más detenidamente.

En este caso, podemos convertir de forma segura valores INT a FLOAT, por lo que queremos decirle a TFDV que use nuestro esquema para inferir el tipo. Hagámoslo ahora.

In [ ]:
options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA, stats_options=options)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

Ahora solo tenemos la característica `tips` (que es nuestra etiqueta) que aparece como una anomalía ("Columna eliminada"). Por supuesto, no esperamos tener etiquetas en nuestros datos de servicio, así que digámosle a TFDV que las ignore.

In [ ]:
# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Specify that 'tips' feature is not in SERVING environment.
tfdv.get_feature(schema, 'tips').not_in_environment.append('SERVING')

serving_anomalies_with_env = tfdv.validate_statistics(
    serving_stats, schema, environment='SERVING')

tfdv.display_anomalies(serving_anomalies_with_env)

## Verificación de la presencia de desviaciones y sesgos

Además de comprobar si un conjunto de datos se ajusta a las expectativas establecidas en el esquema, TFDV también ofrece funciones que permiten detectar la desviación y el sesgo. TFDV ejecuta esta comprobación al comparar las estadísticas de los distintos conjuntos de datos basándose en los comparadores de desviación/sesgo especificados en el esquema.

### Desviación

La detección de desviaciones se admite para características categóricas y entre intervalos de datos consecutivos (es decir, entre el intervalo N y el intervalo N+1), como entre diferentes días de datos de entrenamiento. Expresamos la desviación en términos de [distancia L-infinito](https://en.wikipedia.org/wiki/Chebyshev_distance) y puede establecer la distancia al umbral para recibir advertencias cuando la desviación sea mayor de lo aceptable. Establecer la distancia correcta suele ser un proceso iterativo que requiere experimentación y conocimiento del dominio.

### Sesgo

TFDV puede detectar tres tipos diferentes de sesgo en sus datos: sesgo de esquema, sesgo de características y sesgo de distribución.

#### Sesgo de esquema

El sesgo de esquema ocurre cuando los datos de entrenamiento y servicio no se ajustan al mismo esquema. Se espera que tanto los datos de entrenamiento como los de servicio sigan el mismo esquema. Cualquier desviación esperada entre los dos (como que la característica de la etiqueta solo esté presente en los datos de entrenamiento, pero no en los de servicio) debe especificarse a través del campo de entornos en el esquema.

#### Sesgo de características

El sesgo de características ocurre cuando los valores de características con los que se entrena un modelo son diferentes de los valores de características que se ven en el momento del servicio. Por ejemplo, esto puede suceder en los siguientes escenarios:

- Una fuente de datos que proporciona algunos valores de características se modifica entre el tiempo de entrenamiento y de servicio.
- Existe una lógica diferente para generar características entre el entrenamiento y el servicio. Por ejemplo, si aplica alguna transformación solo en una de las dos rutas de código.

#### Sesgo de distribución

El sesgo de distribución ocurre cuando la distribución del conjunto de datos de entrenamiento es significativamente diferente de la distribución del conjunto de datos de servicio. Una de las causas clave del sesgo en la distribución es el uso de código diferente o de diferentes fuentes de datos para generar el conjunto de datos de entrenamiento. Otra razón es un mecanismo de muestreo defectuoso que elige una submuestra no representativa de los datos de servicio para entrenar.

In [ ]:
# Add skew comparator for 'payment_type' feature.
payment_type = tfdv.get_feature(schema, 'payment_type')
payment_type.skew_comparator.infinity_norm.threshold = 0.01

# Add drift comparator for 'company' feature.
company=tfdv.get_feature(schema, 'company')
company.drift_comparator.infinity_norm.threshold = 0.001

skew_anomalies = tfdv.validate_statistics(train_stats, schema,
                                          previous_statistics=eval_stats,
                                          serving_statistics=serving_stats)

tfdv.display_anomalies(skew_anomalies)

En este ejemplo, vemos cierta desviación, pero está muy por debajo del umbral que establecimos.

## Cómo congelar el esquema

Ahora que revisamos y seleccionamso el esquema, lo almacenaremos en un archivo para reflejar su estado "congelado".

In [ ]:
from tensorflow.python.lib.io import file_io
from google.protobuf import text_format

file_io.recursive_create_dir(OUTPUT_DIR)
schema_file = os.path.join(OUTPUT_DIR, 'schema.pbtxt')
tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

## Cuando usar TFDV

Es fácil pensar que TFDV solo se aplica al inicio de su canalización de entrenamiento, como hicimos aquí, pero en realidad tiene muchos usos. Aquí hay algunos más:

- Validación de nuevos datos para inferencias con el fin de asegurarse de que no haya empezado a recibir características erróneas de repente.
- Validación de nuevos datos para la inferencia para asegurarse de que el modelo se haya entrenado en esa parte de la superficie de decisión.
- Validación de los datos después de haberlos transformado y haber realizado ingeniería de características (probablemente con [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started)) para comprobar que todo se haya ejecutado correctamente.